In [ ]:
import json
with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/cycle/main.json') as f:
    datas = json.load(f)
print(len(datas))
datas['120']

In [ ]:
import json
import openai
import time
# openai.api_type = "azure"
from pathlib import Path
from tqdm import tqdm
openai.api_type = "azure"
openai.api_base = "https://new-llm.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "082b19bc29364b1bb39d2d9fb9b757d4"
def chatGPT_test(text, content=None):

    prompt = [{"role": "system", "content": 'You are a helpful assistant that are good at providing detailed explanations to solve graph reasoning tasks.'},
        {"role": "user", "content": "{}".format(text)}]
    i = 0
    while True:
        try:
            if i>=3:
                break
            completion = openai.ChatCompletion.create(
                engine="gpt-35-turbo",
                messages = prompt,
                temperature=0.0,
                max_tokens=300,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                stop='\n')
            break
        except Exception as e:
            i+=1
            tqdm.write(str(e))
            tqdm.write("Retrying...")
            # import time
            time.sleep(30)
    time.sleep(10.0)
    return completion["choices"][0]["message"]["content"]
with open("/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/prompt/" + "CoT-prompt.txt", "r") as f:
    exemplar = f.read()

with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/train.json') as f:
    datas = json.load(f)

gen_datas_jsonl = '/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/chatgpt/_generate_train.json'
   
start_index = (
        len(open(gen_datas_jsonl).readlines()) if gen_datas_jsonl.exists() else 0
    )
print(f"start_index: {start_index}")
    
for key, value in tqdm(datas.items()):
    
    if int(key) <= start_index:
        continue
    
    query = value['question']
    instruct = query.split('\n')[0] + " Begin with '###' to give your final conclusion."
    query = query.replace(query.split('\n')[0], instruct)
    # query += "\nLet's think step by step:"
    inputs = exemplar + '\n\n\n' + query
    try:
        response = chatGPT_test(inputs)
        new_data = dict()
        value['chatgpt_response'] = response
        new_data[key] = value
        with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/chatgpt/_generate_train.json', "a") as w:
            w.write(json.dumps(new_data, ensure_ascii=False) + '\n')
            
        if 'yes' in value['answer'] and 'yes' in response.split('###')[-1].lower():
            with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/chatgpt/_generate_train_correct.json', "a") as w:
                w.write(json.dumps(new_data, ensure_ascii=False) + '\n')
                
        elif 'no' in value['answer'] and 'no' in response.split('###')[-1].lower():
            with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/chatgpt/_generate_train_correct.json', "a") as w:
                w.write(json.dumps(new_data, ensure_ascii=False) + '\n')
        else:
            with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/connectivity/chatgpt/_generate_train_wrong.json', "a") as w:
                w.write(json.dumps(new_data, ensure_ascii=False) + '\n')
    except:
        continue

In [3]:
with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/GPT4_Gen/gpt4_nlg_sample3_res.json') as f:
    datas = f.readlines()
import json
json.loads(datas[0])
len(datas)
# merge_ = {}
# for data in datas:
#     data = json.loads(data)
#     if data['query'] not in merge_:
#         merge_[data['query']].append()

18064

In [2]:
tasks = ['connectivity', 'cycle','flow', 'GNN', 'hamilton', 'matching','shortest_path', 'topology']
tasks_train_samples = {task:[] for task in tasks}
tasks_test_samples = {task:[] for task in tasks}
import json
with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/GPT4_Gen/gpt4_nlg_sample3_res.json') as f:
    datas = f.readlines()
from tqdm import tqdm
for task in tasks:
    with open(f'/cpfs/user/chennuo/CN/NLGraph/NLGraph/{task}/train.json') as f:
        t_datas = json.load(f)
    t_len = len(t_datas)
    for key, data in tqdm(t_datas.items()):
        response = data['answer']    
        query = data['question']
        for data in datas:
            data = json.loads(data)
            if data['query'] == query:
                tasks_train_samples[task].append(data)
                # break
            
    with open(f'/cpfs/user/chennuo/CN/NLGraph/NLGraph/{task}/test.json') as f:
        t_datas = json.load(f)
    t_len = len(t_datas)
    for key, data in tqdm(t_datas.items()):
        response = data['answer']    
        query = data['question']
        for data in datas:
            data = json.loads(data)
            if data['query'] == query:
                tasks_test_samples[task].append(data)
                # break
                


100%|██████████| 135/135 [00:41<00:00,  3.29it/s]


In [3]:
len(tasks_train_samples['connectivity'])

5588

In [4]:
correct_samples  = {task:[] for task in tasks}
import re
def extract_last_num(text: str) -> float:
    text = re.sub(r"(\d),(\d)", "\g<1>\g<2>", text)  # 处理形如 123,456
    res = re.findall(r"(\d+(\.\d+)?)", text)  # 匹配 123456.789
    if len(res) > 0:
        num_str = res[-1][0]
        return float(num_str)
    else:
        return 0.0


for key, value in tasks_train_samples.items():
    
    if key in ['cycle', 'connectivity']:
        querys = []
        for i, v in enumerate(value):
            truth = v['response']
            v['id'] = i
            if 'yes' in truth.lower() and 'yes' in v['dv3_response'].split('###')[-1].lower():
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
                # else:
                    
            elif 'no' in truth.lower() and 'no' in v['dv3_response'].split('###')[-1].lower():
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
                
    elif key == 'flow':
        querys = []
        for i, v in enumerate(value):
            truth = v['response']
            v['id'] = i
            t_num = extract_last_num(truth)
            p_num = extract_last_num(v['dv3_response'].split('###')[-1])
            if abs(t_num - p_num) < 1e-2:
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
                # correct_samples[key].append(v)
                
    elif key == 'hamilton':
        querys = []
        for i, v in enumerate(value):
            truth = v['response'].split('be: ')[-1].strip('.')
            v['id'] = i
            if truth in v['dv3_response']:
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
                # correct_samples[key].append(v)
                
    elif key == 'matching':
        querys = []
        for i, v in enumerate(value):
            truth = v['response'].split('\n')[-1]
            v['id'] = i
            t_num = extract_last_num(truth)
            p_num = extract_last_num(v['dv3_response'].split('###')[-1])
            if abs(t_num - p_num) < 1e-2:
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
                
    elif key == 'shortest_path':
        querys = []
        for i, v in enumerate(value):
            truth = v['response'].split('is')[-1].split('with')[0].strip(' ')
            v['id'] = i
            # t_num = extract_last_num(truth)
            if truth in v['dv3_response']:
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
                
    elif key == 'topology':
        querys = []
        for i, v in enumerate(value):
            truth = v['response'].split('is: ')[-1].strip(' ').strip('.')
            v['id'] = i
            # t_num = extract_last_num(truth)
            if truth in v['dv3_response']:
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])
            # if i <=5:
            #     print(truth)
            #     print(v['dv3_response'])
            #     print('&&&&&&&&&')
                
    elif key == 'GNN':
        querys = []
        for i, v in enumerate(value):
            truths = v['response'].split('\n')[1:-1]
            v['id'] = i
            # t_num = extract_last_num(truth)
            flag = True
            for truth in truths:
                if truth not in v['dv3_response'].split('###')[-1]:
                    flag = False
                    break
            if flag:
                if v['query'] not in querys:
                    correct_samples[key].append(v)
                    querys.append(v['query'])


In [5]:
for key, value in correct_samples.items():
    print(key)
    print(len(value))

connectivity
1813
cycle
780
flow
83
GNN
70
hamilton
14
matching
416
shortest_path
271
topology
46


In [11]:
for key, value in correct_samples.items():
    print(key)
    print(len(value))

connectivity
1697
cycle
592
flow
45
GNN
63
hamilton
6
matching
393
shortest_path
200
topology
35


In [ ]:
for key, value in  tasks_train_samples.items():
    print(key)
    print(len(value))

In [5]:

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

with open('/cpfs/user/chennuo/CN/XMATH-LLM/data/NLGGraph/train_v2.json', 'w')  as writer, open('/cpfs/user/chennuo/CN/XMATH-LLM/data/NLGGraph/test_v2.json', 'w')  as writer1:
   for key, value in correct_samples.items():
        # print(key)
        # print(len(value)) 
        for i, v in enumerate(value):
            new_data = dict()
            query = v['query'].split('\nA:')[0].replace('\nQ:','\n###Question:')
            new_data['prompt'] = PROMPT_DICT["prompt_no_input"].format(instruction=query)
            new_data['chosen'] = v['dv3_response'].split('Possible answer:\n\n')[-1]
            new_data['reject'] = "I DO NOT KNOW."
            if i <=1:
                writer1.write(json.dumps(new_data, ensure_ascii=False) + '\n')
            else:
                writer.write(json.dumps(new_data, ensure_ascii=False) + '\n')

In [8]:
import numpy as np
ans = np.load('/cpfs/user/chennuo/CN/NLGraph/NLGraph/hamilton/graph/hard/log/20230501---17-44-CoT/answer.npy')
res = np.load('/cpfs/user/chennuo/CN/NLGraph/NLGraph/hamilton/graph/hard/log/20230501---17-44-CoT/res.npy')

In [12]:
import numpy as np
stores = np.load('/cpfs/user/chennuo/dsChatLLama/test_data/results/retrieval/sentence_embeddings.npy', allow_pickle=True)
# stores['vtD61B1EE5AB3D4E54A764481B2D05B8FC-ac49A5EB7BA51D42479C149082AF731293']

In [15]:
stores.item()['vtD61B1EE5AB3D4E54A764481B2D05B8FC-ac49A5EB7BA51D42479C149082AF731293']

tensor([[ 0.5084, -0.2955, -0.8260,  ...,  0.4785,  0.1249, -0.2732],
        [ 1.1506,  0.4685, -0.8244,  ...,  0.3458,  0.3433, -0.3221],
        [ 0.1429, -0.5485, -0.3346,  ...,  0.7254, -1.0859, -0.4258],
        ...,
        [ 0.9380, -0.0750, -1.1993,  ...,  0.9142, -0.4972, -0.6364],
        [-0.4368,  0.5446, -0.8948,  ...,  0.2306,  0.6288,  0.3295],
        [ 1.0552,  0.3792, -1.2823,  ...,  1.0564,  0.5365, -0.2187]])

In [15]:
import json
with open('/cpfs/user/chennuo/CN/NLGraph/NLGraph/hamilton/test.json') as f:
    datas = json.load(f)
datas['0']

{'question': 'In an undirected graph, (i,j) means that node i and node j are connected with an undirected edge.\nThe nodes are numbered from 0 to 8, and the edges are: (0,4) (0,2) (0,6) (0,7) (0,1) (1,5) (2,3) (2,6) (2,5) (3,4) (3,7) (4,7) (4,6) (5,6) (5,7) (6,8) (7,8)\nQ: Is there a path in this graph that visits every node exactly once? If yes, give the path. Note that in a path, adjacent nodes must be connected with edges.\nA:',
 'answer': 'Yes. The path can be: 0,1,5,7,8,6,4,3,2',
 'difficulty': 'easy'}